File: stocks.ipynb\
Author: alexkobz\
Date: 19-11-2024\
Modified: 31-01-2025 (DSRFU-351)\
Task: DSRFU-277

#### Шаг 0. Импорт и получение токена

In [1]:
import asyncio
import requests
import pandas as pd
import numpy as np
import nest_asyncio; nest_asyncio.apply()
from datetime import datetime as dt, date, timedelta
from functions.divide_chunks import divide_chunks
from math import ceil
from functions.get_date import last_day_month, last_day_month_str, last_work_date_month, last_work_date_month_str

# последний день месяца
REPORT_DATE = pd.to_datetime(last_day_month)
REPORT_DATE_STR = last_day_month_str
# увеличиваем количество столбцов и строк для просмотра
pd.set_option('display.max_rows', 200, 'display.max_columns', 200)

In [2]:
from rudata.RuDataDF import RuDataDF, RuDataRequest
RuDataRequest.set_headers()

Getting token started
Getting token finished


#### Шаг 1. Загрузка stage таблиц 

[FintoolReferenceData](https://docs.efir-net.ru/dh2/#/Info/FintoolReferenceData?id=post-fintoolreferencedata) - Получить расширенный справочник по финансовым инструментам.

In [3]:
FintoolReferenceData = RuDataDF("FintoolReferenceData").df
FintoolReferenceData.head()

FintoolReferenceData started
FintoolReferenceData finished. FintoolReferenceData shape (102423, 186)


,facevalue_m,faceftname_m,sumissuevol_m,summarketvol_m,numcoupons_m,nrdcode,regcode_m,regdate_m,regdistdate_m,begdistdate_m,enddistdate_m,endmtydate_m,daysall_m,fullname_nrd,rpregorg_nrd,rpstate_nrd,issuenumber,currentfacevalue_nrd,isqualified_nrd,bondseries,islombardcbr_nrd,numcoupons_nrd,facevalue_nrd,faceftname_nrd,summarketvol_nrd,regcode_nrd,regdate_nrd,regorg_nrd,begdistdate_nrd,daysall_nrd,enddistdate_nrd,endmtydate_nrd,regdistdate_nrd,sumissuevol_nrd,drqty,isincodebase_nrd,nrdcompcode,shqty,summarketvolorig_nrd,issuername_nrd,planmtydate_nrd,firstcoupondate_nrd,depositary_nrd,registrar_nrd,placeprice_nrd,micexlist_nrd,minlot_nrd,formarketbonds_nrd,registraracctype_nrd,registraracctypedate_nrd,facialacc_nrd,acc_open_date_nrd,fraction_nrd,specialized_depo_nrd,cmp_code_nsd_issuer_nrd,cmp_code_nsd_managing_nrd,ratetypenamerus_nrd,ratetypenameeng_nrd,coupontypename_nrd,base_month,base_year,basis_nrd,sectypenamerus_nrd,sectypenameeng_nrd,secformnamerus_nrd,secformnameeng_nrd,sectypenamebr_nrd,secstaterus_nrd,secstateeng_nrd,placementtype_nrd,share_category_name_nrd,share_category_name_en_nrd,dr_category_name_nrd,dr_category_name_en_nrd,company_state_name_nrd,rate_type_id,cp_type_id,coupon_period_base_id,sec_type_id,sec_form_id,sec_state_id,placement_type_id,share_category_id,dr_category_mn,company_state_mn,sector4212u,sna2008,fullname_en_nrd,seniorityname,fintoolid,moex_code,moex_name,fullname,isin144a,isincode,nickname,progfintoolid,regcode,regdate,regorg,amortisedmty,basis,bondstructuralpar,cfi,coupontype,fintooltype,guaranteeamount,guaranteetype,haveoffer,isconvertible,isguaranteed,maturitygroup,securitization,accruedintcalctype,country,couponperyear,faceftname,facevalue,guarantval,havedefault,numcoupons,status,sumissueval,sumissuevol,summarketval,summarketvol,borrowername,borrowerokpo,borrowersector,borroweruid,borrowerinn,borrowercountry,borrowerregionname,issuername,issuerokpo,issuersector,issueruid,issuerinn,issuercountry,issuerregionname,assetssector,numguarantors,begdistdate,begmtydate,daysall,enddistdate,endmtydate,privatedist,regdistdate,securitytype,securitykind,nominaltype,update_time,isregion,nsd_service,raterevisionsperyear,floatratename,issubordinated,note,listing_level,havecovenant,haverepayment,haveindexedfv,numdecimals,couponperiod,ismatched,firstcoupondate,couponrate,iscomplicated,available_for_unqualified,sec_type_br_code,original_country,qualified_test_id,qualified_test_id_tm,base_asset_fintoolid,isqualified,listing_level_spb,have_mm_moex,have_mm_spb,beg_book_date,seniority,tradesite,current_coupontype,profit_type,counter,rn
0,2.0,2.0,NaN,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,209185,None,None,"Ventoux CCM Acquisition Corp, Ordinary Shares",None,US92280L1017,Ventoux CCM -US92280L1017,NaN,None,None,None,NaN,None,None,None,None,Акция,None,None,NaN,0.0,NaN,None,None,NaN,US,NaN,USD,0.0001,NaN,NaN,NaN,В обращении,NaN,NaN,NaN,NaN,None,None,None,NaN,None,None,None,Ventoux CCM Acquisition,None,None,120453.0,None,US,None,None,NaN,None,None,NaN,None,None,0.0,None,Обыкн-ая,Неопределен,None,2023-10-12T00:25:07,NaN,Нет информации,NaN,None,NaN,None,NaN,None,None,None,NaN,NaN,None,None,NaN,None,False,None,US,1000,1000,NaN,None,NaN,False,False,None,NaN,102.0,None,None,102423,82863
1,2.0,2.0,NaN,NaN,NaN,None,0.0,0.0,0.0,0.0,0.0,NaN,NaN,None,None,None,None,NaN,NaN,None,NaN,NaN,NaN,None,NaN,None,None,None,None,NaN,None,None,None,NaN,NaN,None,None,NaN,NaN,None,None,None,None,None,NaN,None,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,N

In [4]:
Calendar = RuDataDF("CalendarV2").df
Calendar.head()

CalendarV2 started
CalendarV2 finished. CalendarV2 shape (903173, 81)


,action_id,fintool_rusbonds_type,issuer_rusbonds_type,original_country,update_date,securityType,privateDist,country,listing_level,type_convertation,literid_typeoperation,act_code,status,status_date,literid_status,pay_decision,unpayed_decision_reason,payed_details,fininstId,issuerName,id,isiNcode,regCode,nickname,nrDcode,finToolID,type,eventID,typeOperation,rateDate,beginPeriod,endPeriod,couponPeriod,couponRate,pay1Bond,moneyFlowVal,eventTotalVol,daysFromDist,execution,payDate,fixDate,mtyDate,mtyPart,offerDate,buyBackPrice,begOrder,endOrder,eventPeriod,convert2FinToolID,coefficient,beginConvDate,endConvDate,defaultID,defaultType,typeEvent,defaultDate,realPayDate,note,eventType,eventDate,beginEventPer,value,faceFTName,pay1Security,moneyFlowPlan,protocolDate,pay1Bond_frac,currencyFTName,paymentCurrencyFTName,revisedPay1Bond,recomendPay1Security,recomendRateDate,recomendFixDate,recomendPayDate,recomendPayDateNominee,fintoolType,dividend_currency,ex_dividend_date,declared_pay_date_nominee,counter,rn
0,158845,M,M,RU,2023-09-04T15:15:26,Корп,False,RU,NaN,None,1098.0,INTR,None,None,NaN,None,None,None,94864.0,РЖД-Развитие вокзалов,None,None,4B02-01-13706-A,РЖД-РВ-1-боб,None,99116,None,10.0,None,None,None,None,182.0,NaN,NaN,NaN,NaN,1820.0,E,None,None,None,NaN,None,NaN,None,None,182.0,None,NaN,None,None,NaN,None,None,None,None,None,CPN,None,None,NaN,RUB,NaN,NaN,None,0.0,RUB,RUB,NaN,0.0,None,None,None,None,Облигация,None,None,None,903173,880467
1,511817,M,M,RU,2024-03-20T20:11:38,Корп,False,RU,NaN,None,911.0,REDM/BN,None,None,NaN,None,None,None,85953.0,Ростелеком Фл Центр,None,None,4B02-05-00194-A,ЦентрТелеком-5-боб,None,49722,None,1.0,OM,2009-11-11T00:00:00,None,None,NaN,NaN,1000.0,1.000000e+09,NaN,NaN,E,None,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,None,None,None,None,MTY,None,None,100.0,RUB,NaN,NaN,None,0.0,None,None,NaN,0.0,None,None,None,None,Облигация,None,None,None,903173,866864
2,511818,M,M,RU,2024-03-20T20:11:38,Корп,False,RU,NaN,None,911.0,REDM/BN,None,None,NaN,None,None,None,85953.0,Ростелеком Фл Центр,None,None,4B02-06-00194-A,ЦентрТелеком-6-боб,None,49723,None,1.0,OM,2009-11-11T00:00:00,None,None,NaN,NaN,1000.0,1.000000e+09,NaN,NaN,E,None,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,None,None,None,None,MTY,None,None,100.0,RUB,NaN,NaN,None,0.0,None,None,NaN,0.0,None,None,None,None,Облигация,None,None,None,903173,866865
3,511819,M,M,RU,2024-03-20T20:11:38,Корп,False,RU,NaN,None,911.0,REDM/BN,None,None,NaN,None,None,None,85953.0,Ростелеком Фл Центр,None,None,4B02-07-00194-A,ЦентрТелеком-7-боб,None,49724,None,1.0,OM,2009-11-11T00:00:00,None,None,NaN,NaN,1000.0,1.000000e+09,NaN,NaN,E,None,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,None,None,None,None,MTY,None,None,100.0,RUB,NaN,NaN,None,0.0,None,None,NaN,0.0,None,None,None,None,Облигация,None,None,None,903173,866866
4,511822,C,C,RU,2024-03-20T20:11:38,Корп,False,RU,NaN,None,911.0,REDM/BN,None,None,NaN,None,None,None,43589.0,Кокс,None,None,4B02-01-10799-F,Кокс-1-боб,None,49810,None,1.0,OM,2009-12-17T00:00:00,None,None,NaN,NaN,1000.0,3.000000e+09,NaN,NaN,E,None,None,None,NaN,None,NaN,None,None,NaN,None,NaN,None,None,NaN,None,None,None,None,None,MTY,None,None,100.0,RUB,NaN,NaN,None,0.0,None,None,NaN,0.0,None,None,None,None,Облигация,None,None,None,903173,866867


In [5]:
AccruedInterestOnDate = RuDataDF("AccruedInterestOnDate").df
AccruedInterestOnDate.head()

AccruedInterestOnDate started
AccruedInterestOnDate finished. AccruedInterestOnDate shape (102423, 14)


,fintoolId,shiftDays,shiftTypeSource,typeCurrency,date,accruedInterest,accruedInterestPct,currentFaceValue,currentFaceValuePct,calculationType,error,faceValueCurrency,accruedInterestCurrency,faceValueLastKnownDate
0,226283,0,Calendar,None,2025-01-31T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-01-31T00:00:00
1,193430,0,Calendar,None,None,NaN,NaN,NaN,NaN,None,Ошибка загрузки описания облигации '193430': Н...,None,None,None
2,174809,0,Calendar,None,2025-01-31T00:00:00,NaN,NaN,250.0,100.0,Typical,None,RUB,RUB,2025-01-31T00:00:00
3,57920,0,Calendar,None,2025-01-31T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-01-31T00:00:00
4,108955,0,Calendar,None,2025-01-31T00:00:00,NaN,NaN,1000.0,100.0,Typical,None,RUB,RUB,2025-01-31T00:00:00


In [6]:
# from __future__ import annotations
# import asyncio
# import aiohttp
# import socket
# import pandas as pd
# from typing import List, Union, Optional, Dict, Any
# from inspect import currentframe
# from time import sleep
# from datetime import datetime

# from functions.get_date import last_day_month
# from logger.Logger import Logger
# from rudata import DocsAPI
# from rudata.RuDataRequest import RuDataRequest
# from rudata.Token import Token
# # from rudata.SavedDF import FintoolReferenceData, Emitents, Calendar
# from functions.divide_chunks import divide_chunks
# from functions.postgres_engine import engine as postrges_engine

# logger = Logger()

# class RuDataDF:
#     engine = postrges_engine
#     report_monthyear: str = last_day_month.strftime("%m%Y")
#     def __init__(self, key=None):
#         if key:
#             self.key: str = key
#             self._url: str = getattr(DocsAPI, key).url
#             self._payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
#             self._requestType: DocsAPI.RequestType = getattr(DocsAPI, key).requestType
#             self._date = None
#             self._ids: List[str | int] = []
#             self._ids_key = None
#             self._from = None
#             self._to = None
#             self._list_json: List[dict] = []
#             self._df: pd.DataFrame = pd.DataFrame()

#     # No usage
#     def set_date(self, date=datetime.today().strftime("%Y-%m-%d")):
#         caller_locals = currentframe().f_back.f_locals
#         keys = [name for name, value in caller_locals.items() if date is value]
#         for key in keys:
#             if key in self._payload:
#                 self._payload[key] = date
#         return self

#     # No usage
#     def set_from(self, from_=datetime.today().strftime("%Y-%m-%d")):
#         caller_locals = currentframe().f_back.f_locals
#         key = [name for name, value in caller_locals.items() if from_ is value][0]
#         self._from = from_
#         self._payload[key] = from_
#         return self

#     # No usage
#     def set_to(self, to=datetime.today().strftime("%Y-%m-%d")):
#         caller_locals = currentframe().f_back.f_locals
#         key = [name for name, value in caller_locals.items() if to is value][0]
#         self._to = to
#         self._payload[key] = to
#         return self

#     # No usage
#     def set_custom(self, value, key):
#         self._payload[key] = value
#         return self

#     async def _get_df(self) -> pd.DataFrame:

#         async def create_execute_tasks(payloads: List[dict]):
#             async with aiohttp.ClientSession(
#                     connector=aiohttp.TCPConnector(limit=DocsAPI.LIMIT,
#                                                    family=socket.AF_INET),
#                     trust_env=True,
#                     timeout=aiohttp.ClientTimeout(7200)
#             ) as session:
#                 for chunk in divide_chunks(payloads, DocsAPI.LIMIT):
#                     tasks: List[asyncio.Task] = [asyncio.create_task(
#                         RuDataRequest(self._url, session).post(payload=payload)
#                     ) for payload in chunk]
#                     resAll: List[List[dict]] = [await task for task in asyncio.as_completed(tasks, timeout=600)]
#                     result: List[dict] = [row for task_res in resAll for row in task_res]
#                     logger.info(f"{len(result)}, {len(self._list_json)}")
#                     if result or self._requestType not in (DocsAPI.RequestType.PAGES,
#                                                             DocsAPI.RequestType.FintoolReferenceData):
#                         self._list_json.extend(result)
#                     else:
#                         return

#         async def get_response_ids(key: Union[str, int]) -> None:
#             if isinstance(self._payload[key], list):
#                 # CompanyRatingsTable
#                 if self._payload[key] and isinstance(self._payload[key][0], DocsAPI.CompanyId):
#                     ids: List[dict] = []
#                     companyIdDict: Dict[str, Any] = self._payload[key][0].__dict__.copy()
#                     for id in self._ids:
#                         companyIdDict["id"] = id
#                         ids.append(companyIdDict.copy())
#                 else:
#                     ids: List[str | int] = self._ids.copy()
#             else:
#                 raise "Ids is invalid"
#             payloads: List[dict] = []
#             payload: Dict[str, Any] = self._payload.copy()
#             for id in divide_chunks(ids, 100):
#                 payload[key] = id
#                 payloads.append(payload.copy())
#             await create_execute_tasks(payloads)

#         async def get_payloads_pages(payloadTMP: dict, key: str) -> List[dict]:
#             pagers: List[dict] = []
#             for pageNum in range(1, 10000):
#                 payloadTMP[key] = pageNum
#                 pagers.append(payloadTMP.copy())
#             return pagers

#         if self._requestType == DocsAPI.RequestType.PAGES:
#             payloads: List[dict] = await get_payloads_pages(self._payload.copy(), "pageNum")
#             asyncio.run(create_execute_tasks(payloads))
#         elif self._requestType == DocsAPI.RequestType.FintoolReferenceData:
#             if isinstance(self._payload["pager"], DocsAPI.Pager):
#                 pagers = await get_payloads_pages(self._payload["pager"].__dict__.copy(), "page")
#                 payloads = []
#                 payload = self._payload.copy()
#                 for pager in pagers:
#                     payload["pager"] = pager
#                     payloads.append(payload.copy())
#                 asyncio.run(create_execute_tasks(payloads))
#         # elif self._requestType == DocsAPI.RequestType.FININSTID:
#         #     self._ids: List[int] = Emitents().get_fininst()
#         #     await get_response_ids("ids")
#         # elif self._requestType == DocsAPI.RequestType.SecurityRatingTable:
#         #     self._ids: List[str] = FintoolReferenceData().get_isin()
#         #     await get_response_ids("ids")
#         elif self._requestType == DocsAPI.RequestType.FINTOOLIDS:
#             self._ids: List[int] = (
#                 pd.read_sql(
#                     f"""
#                     SELECT DISTINCT fintoolid
#                     FROM "FintoolReferenceData"
#                     WHERE report_monthyear = '{RuDataDF.report_monthyear}'
#                     """
#                     , self.engine
#                 )['fintoolid']
#                 .to_list()
#             )
#             await get_response_ids("fintoolIds")
#         elif self._requestType == DocsAPI.RequestType.REGULAR:
#             async with aiohttp.ClientSession(
#                     connector=aiohttp.TCPConnector(limit=DocsAPI.LIMIT,
#                                                    family=socket.AF_INET),
#                     trust_env=True,
#                     timeout=aiohttp.ClientTimeout(3600)
#             ) as session:
#                 result: List[Optional[dict]] = await RuDataRequest(self._url, session).post(self._payload)
#             self._list_json.extend(result)

#         return pd.DataFrame(self._list_json)

#     @property
#     @Logger.logDF
#     def df(self) -> pd.DataFrame:
#         if Token.instance is None:
#             RuDataRequest.set_headers()

#         self._df: pd.DataFrame = asyncio.run(self._get_df())

#         # if self.key == "FintoolReferenceData":
#         #     FintoolReferenceData.instance = self._df
#         # elif self.key == "Emitents":
#         #     Emitents.instance = self._df
#         # elif self.key == "CalendarV2":
#         #     Calendar.instance = self._df

#         sleep(1)
#         return self._df

#     @df.setter
#     def df(self, value) -> None:
#         self._df = value

In [7]:
FloaterData = RuDataDF("FloaterData").df
FloaterData.head()

FloaterData started
FloaterData finished. FloaterData shape (1792, 10)


,fintoolId,beg_period,end_period,start_point,model_type,date_pub_n,date_pub_type,date_fix_n,date_fix_type,bases
0,153053,2025-11-18T00:00:00,2026-05-18T00:00:00,B,single,NaN,None,2.0,W,"[{'base_no': 4, 'calc_type': 'FIX', 'base_fint..."
1,165757,2027-12-03T00:00:00,2028-12-03T00:00:00,B,single,NaN,None,2.0,W,"[{'base_no': 2, 'calc_type': 'FIX', 'base_fint..."
2,164125,2029-04-02T00:00:00,2029-10-02T00:00:00,B,max,NaN,None,2.0,W,"[{'base_no': 2, 'calc_type': 'FIX', 'base_fint..."
3,164125,2034-04-02T00:00:00,2034-10-02T00:00:00,B,max,NaN,None,2.0,W,"[{'base_no': 4, 'calc_type': 'FIX', 'base_fint..."
4,145298,2019-09-27T00:00:00,2045-09-20T00:00:00,B,single,NaN,None,5.0,W,"[{'base_no': 2, 'calc_type': 'PER', 'base_fint..."


In [8]:
ExchangeTree = RuDataDF("ExchangeTree").df
ExchangeTree.head()

ExchangeTree started
ExchangeTree finished. ExchangeTree shape (239, 10)


,id,id_parent,shortname_rus,shortname_eng,fullname_rus,fullname_eng,sort_order,update_date,counter,rn
0,144,63,Нал,Cash,Наличная валюта,Cash,0,2020-12-28T17:03:09,239,1
1,270,63,RUDIP 2,RUDIP 2,RUDIP 2,RUDIP 2,1,2022-10-31T19:41:24,239,2
2,84,83,США,US,Макроиндикаторы США,USA macroindicators,1,2020-12-28T17:03:09,239,3
3,85,83,РФ,RU,Макроиндикаторы Россия,Russia macroindicators,2,2020-12-28T17:03:09,239,4
4,2,1,РТС,RTS,РТС индексы,RTS indexes,2,2020-12-28T17:03:09,239,5


In [9]:
Emitents = RuDataDF("Emitents").df
Emitents.head()

Emitents started
Emitents finished. Emitents shape (41577, 91)


,id_emitent,fininstid,shortname_rus,shortname_eng,inn,okpo,ogrn,code,sector,legal_address,phone,www,update_date,fullname_rus_nrd,fullname_eng_nrd,shortname_rus_nrd,shortname_eng_nrd,company_type_short_name,state_reg_num,state_reg_date,state_reg_name,egrul_date,egrul_organ_name,country,lei_code,bik,fax,e_mail,credit_cmp,is_bank_4_non_resident,swift,country_oksm,country_oksm_cbr,country_name_rus,country_name_eng,region_inn,region_soato,region_name,fullname_rus,sparkid,br_fcsm_reg_code,capital,capital_unit,okved,oecd_lvl,oktmo,okato,post_address,okopf,okogu,have_rating,have_risk,is_finorg,is_control_by_finorg,is_gov_or_cb,on_behalf_of_state,is_subjectrf,is_cis_reg,issuer_nrd,tin,kpp,state_reg_number,market_id_nrd,market_name_rus_nrd,market_name_eng_nrd,sp_rx_entity_id,sp_shortname,mds_shortname,mds_org_id,fch_id,fch_shortname,is_monopoly,is_strategic,ifo_list,isregion,sic,sector4212u,sna2008,reg_code,reg_date,reg_org,note,primary_reg_date,inn_fle,other_tin,other_tin_name,okfs_id,okfs_name,is_branch,counter,rn
0,93627,119953,Brazil Minerals,None,None,None,None,XX030692,None,"C/O Incorp Services Inc., 3773 Howard Hughes P...",None,None,2023-06-16T13:15:10.542755,None,None,None,None,Inc US,None,None,None,None,None,USA,5493006FIK0ZLW06RD76,None,None,None,None,None,None,840.0,840.0,США,United States of America,None,None,None,Brazil Minerals Inc,NaN,None,0.0,None,None,0,None,None,None,None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,None,None,None,RA999999,None,None,None,None,None,None,None,False,41577,33095
1,93955,120161,SEB Denmark,None,None,None,None,XX030895,None,Bernstorffsgade 50 1577 Copenhagen V Denmark,None,https://sebgroup.com/about-us/our-business/our...,2022-11-01T12:41:00.457749,None,None,None,None,Branch DK,None,None,None,None,None,DNK,None,None,None,None,None,None,None,208.0,208.0,ДАНИЯ,Denmark,None,None,None,"Skandinaviska Enskilda Banken AB, Denmark Branch",NaN,None,0.0,None,None,0,None,None,None,None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,False,41577,33278
2,94034,120162,Midclear,None,None,None,None,XX030906,Финанс.сервисы,"PO Box 11-7971, 16th Floor, Aresco Center, San...",None,None,2024-05-24T18:42:04,None,None,None,None,SAL LB,None,None,None,None,None,LBN,549300OMI3WHG0X9PA58,None,None,None,None,None,None,422.0,422.0,ЛИВАН,Lebanon,None,None,None,Midclear S.A.L.,NaN,None,0.0,None,None,7,None,None,None,None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,None,None,None,RA999999,None,None,None,None,None,None,None,False,41577,33279
3,89678,117750,Green Trace,None,40103309320,None,None,XX028690,None,"Biekensalas str. 6, LV-1004, Riga, Latvia",None,None,2022-02-11T15:05:25,None,None,None,None,SIA (ООО) LV,None,None,None,None,None,LVA,5493002KC87ZH6T8VD75,None,None,None,None,None,None,428.0,428.0,ЛАТВИЯ,Latvia,None,None,None,Green Trace SIA,NaN,None,0.0,None,None,0,None,None,None,None,None,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,None,LV40103309320,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,S2,40103309320,2010-07-26T00:00:00,None,None,None,None,None,None,None,None,False,41577,31079
4,90039,117890,Dongwon Systems,None,1238115163,None,None,XX028776,Прочие,"68, Mabang-ro Seocho-gu, Seoul, South Korea",None,https://www.dongwonsystems.co.kr,2024-12-02T16:12:06.792847,None,None,None,None,None,None,None,None,None,None,KOR,None,None,None,None,None,None,None,410.0,410.0,КОРЕЯ Южная,Korea,None,None,None,Dongwon Systems Corporation,NaN,None,0.0,None,None,0,None,None,None,None,None,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,None,123-81-15163,None,None,NaN,None,None,None,None,None,None,None,None,0,0,None,NaN,None,None,None,123-81-15163,None,None,None,None,None,134111-0011712,Corp Num KR,None,None,False,41577,31211


### BONDS

#### Шаг 2. Предобработка таблиц

In [10]:
TransformedFintoolReferenceData = FintoolReferenceData[FintoolReferenceData['fintooltype']=='Облигация']\
.copy()[[
    'isincode', 'fintoolid', 'status', 'nickname', 'issuername', 'issuerinn', 'borrowername', 'borrowerinn',
    'fintooltype', 'facevalue', 'begdistdate', 'faceftname', 'nominaltype', 'coupontype', 'floatratename'
]]\
.drop_duplicates(subset=['isincode'])\
.rename(columns={
    'isincode': "isin",
    'fintoolid': "fintool",
    'status': "status",
    'nickname': "name",
    'issuername': "issuer",
    'issuerinn': "issuer_INN",
    'borrowername': "borrower",
    'borrowerinn': "borrower_INN",
    'fintooltype': "type",
    'facevalue': "face_value",
    'begdistdate': "issue_date",
    'faceftname': "currency",
    'nominaltype': "principal_type",
    'coupontype': "coupon_type",
    'floatratename': "rate"
})
TransformedFintoolReferenceData.set_index('isin', drop=True, inplace=True)
TransformedFintoolReferenceData['issue_date'] = pd.to_datetime(TransformedFintoolReferenceData['issue_date'])

Для обновления индексируемого номинала

In [11]:
INDEX_TYPES = ['Инфляционно-Индексируемый', 'Индексируемый', 'Инфляц-Индекс+Амортизация', 
               'Индексация по активу', 'Индексац.(актив)+ Аморт.', 'Инфляционно-Индексируемый']
index_types_isins = TransformedFintoolReferenceData[TransformedFintoolReferenceData['principal_type'].isin(INDEX_TYPES)].index

In [12]:
TransformedAccruedInterestOnDate = AccruedInterestOnDate.copy().dropna(subset=["currentFaceValue"])
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate\
.merge(FintoolReferenceData[['fintoolid', "isincode"]], how='left', left_on='fintoolId', right_on='fintoolid')\
.drop(columns=['fintoolid'])\
.rename(columns={
    "isincode": "isin", 
    'currentFaceValue': 'face_value', 
    'faceValueCurrency': 'currency'
})
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate[['isin', 'face_value', 'currency']].set_index('isin')

#Добавление загруженных данных в результирующий датафрейм
TransformedAccruedInterestOnDate = TransformedAccruedInterestOnDate[~TransformedAccruedInterestOnDate.index.duplicated()]
TransformedAccruedInterestOnDateIndexed = TransformedAccruedInterestOnDate[
    TransformedAccruedInterestOnDate.index.isin(index_types_isins)
]
TransformedFintoolReferenceData.update(TransformedAccruedInterestOnDateIndexed)

In [13]:
TransformedFloaterData = FloaterData.copy()
try:
    TransformedFloaterData = pd\
    .concat([TransformedFloaterData, pd.DataFrame([base[0] for base in FloaterData.bases])], axis=1)\
    .drop(columns=['bases'])
except:
    pass
TransformedFloaterData = TransformedFloaterData\
.merge(FintoolReferenceData[['fintoolid', "isincode"]], how='left', left_on='fintoolId', right_on='fintoolid')\
.drop(columns=['fintoolid'])\
.reset_index(drop=True)
TransformedFloaterData['beg_period'] = pd.to_datetime(TransformedFloaterData['beg_period'])
TransformedFloaterData['end_period'] = pd.to_datetime(TransformedFloaterData['end_period'])
TransformedFloaterData['premium_to_base'] /= 10000

In [14]:
TransformedCalendar = Calendar.copy()
TransformedCalendar.loc[TransformedCalendar['typeOperation'].isin(['O', 'N', 'P']), 'eventType'] = 'PUT'
TransformedCalendar['eventDate'] = pd.to_datetime(TransformedCalendar['eventDate'])
# TransformedCalendar['rateDate'] = pd.to_datetime(TransformedCalendar['rateDate'])
TransformedCalendar = TransformedCalendar[~TransformedCalendar['eventDate'].isna()]
TransformedCalendar.loc[~TransformedCalendar['revisedPay1Bond'].isna(), 'pay1Bond'] = \
TransformedCalendar.loc[~TransformedCalendar['revisedPay1Bond'].isna(), 'revisedPay1Bond']
TransformedCalendar['Currency_temp'] = TransformedCalendar['faceFTName']
TransformedCalendar.loc[~TransformedCalendar['currencyFTName'].isna(), 'Currency_temp'] = \
TransformedCalendar.loc[~TransformedCalendar['currencyFTName'].isna(), 'currencyFTName']
TransformedCalendar.loc[(~TransformedCalendar['paymentCurrencyFTName'].isna())&(~TransformedCalendar['revisedPay1Bond'].isna()), 'Currency_temp'] = \
TransformedCalendar.loc[(~TransformedCalendar['paymentCurrencyFTName'].isna())&(~TransformedCalendar['revisedPay1Bond'].isna()), 'paymentCurrencyFTName']
TransformedCalendar = TransformedCalendar[TransformedCalendar['eventType'].isin(['CPN', 'MTY', 'PUT'])] # если нужны оферты колл, нужно аккуратно подойти к выбору typeOperation
TransformedCalendar['value'] /= 100

In [15]:
%%time
import aiohttp
import socket
from typing import List, Union, Optional, Dict, Any
from rudata import DocsAPI
from time import sleep
from logger.Logger import Logger

key = 'CurrencyRate'
url: str = getattr(DocsAPI, key).url
payload: Dict[str, Any] = getattr(DocsAPI, key)().payload()
currencies_rudata = []
curr = TransformedCalendar[['Currency_temp', 'faceFTName']].drop_duplicates()
curr = curr[curr['Currency_temp'] != curr['faceFTName']].dropna().reset_index(drop=True)

def get_currency_rate(row):
    try:
        response = requests.post(url, json={'from': row['Currency_temp'], 'to': row['faceFTName'], 'date': REPORT_DATE_STR}, headers=RuDataRequest.headers)
    except Exception as e:
        sleep(10)
        Logger.exception(e)
        response = get_currency_rate(row)
    return response
    
for _, row in curr.iterrows():
    response = get_currency_rate(row)
    result: dict = response.json()
    result["from"] = row['Currency_temp']
    result["to"] = row['faceFTName']
    currencies_rudata.append(result)
    sleep(1)
CurrencyRate = pd.DataFrame(currencies_rudata)
CurrencyRate.rename(columns={"rate": "FX"}, inplace=True)
CurrencyRate.head()

CPU times: user 380 ms, sys: 27.5 ms, total: 408 ms
Wall time: 18.2 s


,FX,endOfDayRate,error,from,to
0,0.010203,None,None,RUB,USD
1,0.009730,None,None,RUB,EUR
2,1.243400,None,None,GBP,USD
3,98.006200,None,None,USD,RUB
4,0.008206,None,None,RUB,GBP


In [16]:
TransformedCalendar = TransformedCalendar.merge(CurrencyRate[["FX", "from", "to"]], how='left', left_on=['Currency_temp', 'faceFTName'], right_on=['from', 'to'])
TransformedCalendar['FX'] = TransformedCalendar['FX'].fillna(1)
TransformedCalendar['CF'] = TransformedCalendar['pay1Bond'] * TransformedCalendar['FX']
currencies = pd.read_sql("""
    SELECT DISTINCT num, currency
    FROM currencies
""", RuDataDF.engine).set_index("currency")["num"]
TransformedCalendar['Currency'] = TransformedCalendar['faceFTName'].map(currencies).fillna(0).astype(np.int32)

In [17]:
TransformedCalendar = TransformedCalendar[[
    'isiNcode', 'finToolID', 'nickname', 'rateDate', 'eventDate', 'eventType', 
    'typeOperation', 'value', 'CF', 'faceFTName', 'note', 'eventTotalVol'
]].rename(columns={
    'isiNcode': "isin",
    'finToolID': "fintool",
    'nickname': "name",
    'rateDate': "rate_date", 
    'eventDate': "payment_date", 
    'eventType': "payment_type", 
    'typeOperation': 'operation_type',
    'value': "rate", 
    'CF': "payment_sum", 
    'faceFTName': "currency", 
    'note': "note",
    'eventTotalVol': 'event_total_vol'
})
TransformedCalendar.dropna(subset="isin", inplace=True)

#### Шаг 3. Корректировка потока

In [18]:
bonds = (
    TransformedCalendar
    .merge(
        TransformedFintoolReferenceData[["face_value", "principal_type", "coupon_type", "issue_date"]], 
        how='left', 
        left_on="isin", 
        right_index=True)\
    .rename(columns={
        "face_value": "nominal"
    })
)
bonds["nominal"] = bonds["nominal"].fillna(0)
bonds["rate"] = bonds["rate"].fillna(0)
bonds["payment_sum"] = bonds["payment_sum"].fillna(0)
bonds['event_total_vol'] = bonds['event_total_vol'].fillna(0)
bonds["name"] = bonds["name"].str.replace('[\n\t\r;,]', '.', regex=True)
bonds["note"] = bonds["note"].str.replace('[\n\t\r;,]', '.', regex=True)
bonds["payment_type_NUM"] = bonds["payment_type"].map({"CPN": 1, "MTY": 2, "PUT": 3})
bonds.sort_values(['isin', 'payment_date', 'payment_type_NUM'], inplace=True)

Обнуляем ставки по плавающим купонам, оцененных после отчетной даты

In [19]:
bonds.loc[(bonds['rate_date']>REPORT_DATE_STR) & (bonds['payment_type']=='CPN') & (bonds['operation_type']=='Плавающий'), ['rate', 'payment_sum']] = np.nan

Заполняем пропущенные ставки купона последним известным

In [20]:
bonds.loc[bonds['payment_type']=='CPN', 'rate'] = np.where(
    bonds.loc[bonds['payment_type']=='CPN', 'rate']<1e-6, 
    np.nan, 
    bonds.loc[bonds['payment_type']=='CPN', 'rate']
)
bonds.loc[bonds['payment_type']=='CPN', 'rate'] = (
    bonds.loc[bonds['payment_type']=='CPN', ['isin', 'rate']]
    .groupby('isin')['rate']
    .ffill()
)

Запоминаем ставку предыдущего купона

In [21]:
bonds['prev_cpn'] = (
    bonds
    .groupby("isin")["rate"]
    .shift(1)
).combine_first(bonds['rate'])

Заполняем разницу между датами выплат

In [22]:
bonds['prev_date'] = (
    bonds
    .groupby('isin')['payment_date']
    .shift(1)
)
bonds['days_diff'] = (bonds['payment_date'] - bonds['prev_date']).dt.days

Заполняем погашенный и непогашенный номинал

In [23]:
bonds["MTY"] = np.where(bonds.payment_type=="MTY", bonds['rate'], 0)
bonds["paid_nominal"] = (
    bonds
    .groupby(["isin"])["MTY"]
    .cumsum()
)
bonds["remain_nominal_no_shift"] = bonds["nominal"]*(1 - bonds["paid_nominal"])
bonds["remain_nominal"] = (
    bonds
    .groupby("isin")["remain_nominal_no_shift"]
    .shift(1)
    .fillna(bonds['nominal'])
)

если у нас perpetual, то есть нет даты погашения, добавляем в конец прогнозных потоков погашение

In [24]:
last_type = (
    bonds
    .groupby('isin')
    .last()
    .reset_index()
)
last_noMTY_isins = last_type[last_type['payment_type']=='CPN']['isin']
last_noMTY = last_type[last_type['isin'].isin(last_noMTY_isins)]
last_noMTY['payment_type'] = 'MTY'
last_noMTY['payment_type_NUM'] = 2
last_noMTY['payment_sum'] = (
    last_noMTY["isin"].map(
        bonds
        .groupby('isin')
        .last()["remain_nominal"]
        .to_dict()
    )
)
bonds = pd.concat(
    [bonds, last_noMTY], axis=0)\
.sort_values(['isin', 'payment_date', 'payment_type_NUM'])\
.reset_index(drop=True)

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_13992/3382992906.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_noMTY['payment_type'] = 'MTY'
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_13992/3382992906.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_noMTY['payment_type_NUM'] = 2
/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_13992/3382992906.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

Корректируем поток для облигаций с индексируемым номиналом

In [25]:
indexed_bonds_isins = bonds[bonds['principal_type']=='Инфляционно-Индексируемый']['isin']
indexed_bonds = bonds[bonds['isin'].isin(indexed_bonds_isins)]
not_indexed_bonds = bonds[~bonds['isin'].isin(indexed_bonds_isins)]
cpn_pay_mask = indexed_bonds['payment_type']=='CPN'
mty_pay_mask = indexed_bonds['payment_type']=='MTY'
indexed_bonds.loc[cpn_pay_mask,'payment_sum'] = indexed_bonds.loc[cpn_pay_mask,'rate']*indexed_bonds.loc[cpn_pay_mask,'nominal']*indexed_bonds.loc[cpn_pay_mask,'days_diff']/365
indexed_bonds.loc[mty_pay_mask,'payment_sum'] = indexed_bonds.loc[mty_pay_mask,'remain_nominal']
bonds = pd.concat([indexed_bonds, not_indexed_bonds], axis=0).sort_values(['isin', 'payment_date', 'payment_type_NUM']).reset_index(drop=True)

оставляем только будущие потоки

In [26]:
bonds = bonds[bonds['payment_date']>REPORT_DATE]

обновляем погащение для индексируемого номинала

In [27]:
bonds.loc[bonds['isin'].isin(index_types_isins), "remain_nominal_no_shift"] = \
bonds.loc[bonds['isin'].isin(index_types_isins), "nominal"]*(1 - bonds.loc[bonds['isin'].isin(index_types_isins), "paid_nominal"])
bonds.loc[bonds['isin'].isin(index_types_isins), "remain_nominal"] = (
    bonds.loc[bonds['isin'].isin(index_types_isins), ['isin', "remain_nominal_no_shift"]]
    .groupby("isin")
    .shift(1)
    .fillna(bonds.loc[bonds['isin'].isin(index_types_isins), 'nominal'])
)
# индексируемый номинал уже обновлен
bonds.loc[bonds['isin'].isin(index_types_isins), "remain_nominal"] = \
bonds.loc[bonds['isin'].isin(index_types_isins), "nominal"]

обрезаем потоки по первую дату оферты

In [28]:
bonds['put'] = (
    bonds['payment_type']
    .eq('PUT')
    .groupby(bonds['isin'])
    .cumsum()
    .groupby(bonds['isin'])
    .shift(1)
    .fillna(0)
)
bonds = bonds[bonds['put']==0]

Удаляем погашенные потоки

In [29]:
bonds = bonds[bonds['remain_nominal']>1e-6]

Деление на флоатеры и обычные облигации

In [30]:
Bonds = bonds[~bonds['fintool'].isin(FloaterData["fintoolId"])]
floaters = bonds[bonds['fintool'].isin(FloaterData["fintoolId"])]

Флоатеры

Джойн лучше сделать в sql

In [31]:
import sqlite3

conn = sqlite3.connect(":memory:") 
TransformedFloaterData.to_sql("TransformedFloaterData", conn, index=False)
floaters.to_sql("floaters", conn, index=False)
Floaters = pd.read_sql_query("""
    SELECT 
        b.*
        , fl.beg_period
        , fl.end_period
        , fl.premium_to_base
        , fl.start_point
    FROM floaters b
    LEFT OUTER JOIN TransformedFloaterData fl ON b.fintool = fl.fintoolId AND b.payment_date >= fl.beg_period AND b.payment_date < fl.end_period
    """
    , conn
)
Floaters['payment_date'] = pd.to_datetime(Floaters['payment_date'])
Floaters.loc[Floaters['rate_date'].isna(), 'rate_date'] = date(1970, 1, 1)
Floaters['rate_date'] = pd.to_datetime(Floaters['rate_date'])

Запоминаем фиксированный купон

In [32]:
floaters_fix = Floaters[(Floaters['payment_type']=='CPN') & (Floaters['operation_type']!='Плавающий')]
floaters_fixed_dict = floaters_fix.groupby('isin').first().to_dict()['rate']
floaters_fix['rate'] = floaters_fix['isin'].map(floaters_fixed_dict)
floaters_float = Floaters[(Floaters['payment_type']=='CPN') & (Floaters['operation_type']=='Плавающий')].reset_index()

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_13992/559201119.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floaters_fix['rate'] = floaters_fix['isin'].map(floaters_fixed_dict)


Плавающие купоны

Если БЛИЖАЙШИЙ после расчетной даты будущий купон не пустой и дата rateDate больше расчетной даты, то делаем его пустым (с последующим заполнением предыдущим купоном)

In [33]:
floaters_float = Floaters[(Floaters['payment_type']=='CPN') & (Floaters['operation_type']=='Плавающий')]

In [34]:
floaters_float.loc[
            floaters_float['rate'].isna(), 'rate'
] = (
    floaters_float.loc[
            floaters_float['rate'].isna(), ['premium_to_base', 'prev_cpn']
    ].max(axis=1)
)

In [35]:
cpn_pay_nan_mask = (floaters_float['payment_sum']<1e-6) | (floaters_float['payment_sum'].isna())
floaters_float['days_diff'] = floaters_float['days_diff'].fillna(floaters_float.groupby('isin')['days_diff'].transform('mean'))
floaters_float.loc[cpn_pay_nan_mask, 'payment_sum'] = (
    floaters_float.loc[cpn_pay_nan_mask, 'rate']*
    floaters_float.loc[cpn_pay_nan_mask, 'remain_nominal']*
    floaters_float.loc[cpn_pay_nan_mask, 'days_diff']/365
)

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_13992/2330993031.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  floaters_float['days_diff'] = floaters_float['days_diff'].fillna(floaters_float.groupby('isin')['days_diff'].transform('mean'))


По той же формуле, где фиксированные купоны неизвестны

In [36]:
floaters_fix.loc[floaters_fix['payment_sum']<1e-6, 'payment_sum'] = (
    floaters_fix.loc[floaters_fix['payment_sum']<1e-6, 'rate']*
    floaters_fix.loc[floaters_fix['payment_sum']<1e-6, 'remain_nominal']*
    floaters_fix.loc[floaters_fix['payment_sum']<1e-6, 'days_diff']/365
)

Погашения

In [37]:
mty_floaters = Floaters[(Floaters['payment_type']!='CPN')]

Конкатенируем купоны и погашения

In [38]:
Floaters = (
    pd.concat(
        [floaters_fix, floaters_float, mty_floaters], axis=0
    ).sort_values(
        ['isin', 'payment_date', 'payment_type_NUM']
    )
)

#### Шаг 4. Объединение облигаций с постоянным и плавающим купоном в один датафрейм

In [39]:
Bonds["is_floater"] = False
Floaters["is_floater"] = True

/var/folders/g4/mpm4njl93p547kbhxbvqlm480000gn/T/ipykernel_13992/4080972275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Bonds["is_floater"] = False


In [40]:
bonds = (
    pd.concat([Bonds, Floaters], axis=0)
    .sort_values(['isin', 'payment_date', 'payment_type_NUM'])
    .reset_index(drop=True)
)

In [41]:
bonds['id_payment'] = bonds.groupby(['isin']).cumcount() + 1
bonds["report_date"] = pd.to_datetime(REPORT_DATE)
bonds["payment_date"] = pd.to_datetime(bonds["payment_date"])

Мэппинг имен и кодов валют

In [42]:
bonds["currency"] = (
    bonds["currency"]
    .map(currencies)
    .fillna(0)
    .astype(np.int32)
)

Добавление времени выгрузки и выбор колонок

In [43]:
bonds = bonds[
    ["isin", "fintool", "name", "payment_date", "payment_type", "rate", "payment_sum", "currency", "report_date"]
]

In [44]:
bonds["export_timestamp"] = dt.now().strftime("%Y%m%d%H%M%S")
bonds.to_parquet(f'./data/Output/cash_flows.parquet', index=False)

### SHARES

#### Шаг 5. Предобработка таблиц

In [45]:
shares = FintoolReferenceData[FintoolReferenceData['fintooltype'].isin(['Акция','Выпуск акции'])].copy()
shares = shares[~shares['isincode'].isna()]

In [46]:
not_dup = shares[~shares.duplicated(subset=['isincode'], keep=False)]
dup = shares[shares.duplicated(subset=['isincode'], keep=False)]
dup = dup[dup['securitykind']=='Осн-ой']
dup['count_not_null'] = dup[['summarketvol','issuerinn','issuercountry','tradesite']].count(axis=1)
dup = dup.sort_values(['isincode', 'count_not_null']).groupby('isincode').last().reset_index()
shares = pd.concat([not_dup, dup], axis=0)

In [47]:
shares = shares.merge(Emitents[['fininstid','country_oksm']], how='left', left_on='issueruid', right_on='fininstid')

In [48]:
exch = ExchangeTree[['id','fullname_rus']].set_index('id')
shares.loc[~(shares['tradesite'].isin(exch.index) | shares['tradesite'].isna()), 'tradesite'] = np.nan
shares.loc[shares['tradesite'].isna()==False, 'tradesite'] = \
shares.loc[shares['tradesite'].isna()==False, 'tradesite'].apply(lambda x: exch.loc[int(x)].values[0])
shares[['is_stock_netorg']] = 1
shares.loc[
    (shares['tradesite'].isna()==False) |
    (shares['listing_level'].isna()==False) |
    (shares['listing_level_spb'].isna()==False), 'is_stock_netorg'] = 0

In [49]:
cal = Calendar[Calendar['eventType']=='CONV'].copy()
shares_ = pd.DataFrame().reindex(columns=shares.columns)
df_ = cal.loc[cal['literid_typeoperation'].isin([1074,1075]), ['finToolID','coefficient']].groupby('finToolID').prod(numeric_only=False)
cal = cal[['finToolID']].drop_duplicates().merge(df_, on='finToolID', how='left')
df = shares.merge(cal, how='left', left_on='fintoolid', right_on='finToolID')
df['report_date'] = REPORT_DATE
#если событий по бумаге не было, коэффициент конвертации 1
df.loc[~df['fintoolid'].isin(cal['finToolID'].values), 'coefficient'] = 1
#рассчитываем долю одной акции в капитале
df.loc[df['summarketvol'].isna(), 'summarketvol'] = df.loc[df['summarketvol'].isna(), 'sumissuevol']
df['summarketval'] = df['summarketvol']*df['facevalue']
df['capital_value'] = df.groupby('fininstid')['summarketval'].transform('sum')
df.loc[df['capital_value']==0, 'capital_value'] = np.nan
df['1_share_in_cap'] = (df['facevalue']/df['coefficient'])/df['capital_value']
#если по разным выпускам эмитента разная валюта, наш расчет неверный, обнуляем
df1 = df.groupby('fininstid')['faceftname'].nunique()
df1 = df1.loc[df1>1]
df.loc[df['fininstid'].isin(df1.index), ['summarketval','capital_value','1_share_in_cap']] = np.nan
shares = pd.concat([shares_, df], axis=0)
shares = shares[~(shares['tradesite'].isna() & shares['country_oksm'].isna() & shares['1_share_in_cap'].isna())]
shares["export_timestamp"] = dt.now().strftime("%Y%m%d%H%M%S")
shares = shares[[
    'fintoolid', 'isincode', 'faceftname', 'facevalue', 'summarketvol', 'issuername',
    'issueruid', 'issuerinn', 'issuercountry', 'securitytype', 'listing_level', 'listing_level_spb',	
    'tradesite', 'country_oksm', 'is_stock_netorg', 'coefficient', 'report_date', 'summarketval', 
    'capital_value', '1_share_in_cap', 'export_timestamp']].reset_index(drop=True)

In [50]:
shares.to_parquet(f'data/Output/stocks_data.parquet', index=False)

#### Шаг 6. Отправка письма

In [51]:
from functions.send_email import main
main("cash_flows.parquet", "stocks_data.parquet")